In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [89]:
#Preparation
Qe = -1
Ve = -1/2 + 2*(0.222246)
Ae = -0.50
Vm = -1/2 + 2*(0.222246)
Am = -0.50
alpha = 1/132.507
GF = 1.16639e-5
MZ = 91.188
GammaZ = 2.4414

XMIN = 0.0
YMIN = 0.0
XMAX = 5.0
YMAX = 5.0

# Function to calculate chi1(s)
def chi1(s):
    kappa = np.sqrt(2) * GF * MZ**2 / (4 * np.pi * alpha)
    return kappa * s * (s - MZ**2) / ((s - MZ**2)**2 + GammaZ**2 * MZ**2)

# Function to calculate chi2(s)
def chi2(s):
    kappa = np.sqrt(2) * GF * MZ**2 / (4 * np.pi * alpha)
    return kappa**2 * s**2 / ((s - MZ**2)**2 + GammaZ**2 * MZ**2)

# Coefficients A0(s) and A1(s)
def A0(s):
    return Qe**2 - 2 * Qe * Ve * Vm * chi1(s) + (Am**2 + Vm**2) * (Ae**2 + Ve**2) * chi2(s)

def A1(s):
    return -4 * Qe * Am * Ae * chi1(s) + 8 * Am * Vm * Ae * Ve * chi2(s)

# Differential cross section function
def diff_cross_section(s, cos_theta):
    return alpha**2 / (4 * s) * (A0(s) * (1 + cos_theta**2) + A1(s) * cos_theta)

print(diff_cross_section(0.5, 1))

5.6951248831327665e-05


In [3]:
s_min, s_max = 10**2, 200**2
cos_theta_min, cos_theta_max = -1, 1

# Sampling differential cross-section over the domain to find the maximum value
sampled_values = []
for _ in range(100000):
    random_s = np.random.uniform(s_min, s_max)
    random_cos_theta = np.random.uniform(cos_theta_min, cos_theta_max)
    value = diff_cross_section(random_s, random_cos_theta)
    sampled_values.append(value)

F_VAL_MAX = max(sampled_values)

            
#Part a
def brute_force(nPoints, seed=None):
    nFunctionEval = 0
    s_rej_method = []
    cos_theta_rej_method = []
    maxWeightEncounteredRej = -1.0e20
    generator = np.random.RandomState(seed=seed)
    
    while len(s_rej_method) < nPoints:
        rr = generator.uniform(size=3)
        random_s = np.random.uniform(s_min, s_max)
        random_cos_theta = np.random.uniform(cos_theta_min, cos_theta_max)
        random_value = np.random.uniform(0, F_VAL_MAX)
        nFunctionEval += 1
        f_val = diff_cross_section(random_s, random_cos_theta)
        if f_val > maxWeightEncounteredRej:
            maxWeightEncounteredRej = f_val
        if f_val > F_VAL_MAX:
            print(
                f" f_val={f_val} exceeds F_VAL_MAX={F_VAL_MAX}, program will now exit"
            )
            exit(99)
        if f_val / F_VAL_MAX > rr[2]:
            s_rej_method.append(random_s)
            cos_theta_rej_method.append(random_cos_theta)
    return (s_rej_method, cos_theta_rej_method, nFunctionEval, maxWeightEncounteredRej)

s_rej_method, cos_theta_rej_method, nFunctionEval_rej, maxWeightEncounteredReJ = brute_force(10000)
print(nFunctionEval_rej)

 f_val=6.696541420322152e-07 exceeds F_VAL_MAX=6.631486904825394e-07, program will now exit
 f_val=6.633291630888075e-07 exceeds F_VAL_MAX=6.631486904825394e-07, program will now exit
 f_val=6.777396135790392e-07 exceeds F_VAL_MAX=6.631486904825394e-07, program will now exit
 f_val=6.684942132608976e-07 exceeds F_VAL_MAX=6.631486904825394e-07, program will now exit
 f_val=6.776847141826566e-07 exceeds F_VAL_MAX=6.631486904825394e-07, program will now exit
 f_val=6.665661788964468e-07 exceeds F_VAL_MAX=6.631486904825394e-07, program will now exit
649218


In [77]:
#part b
def setup_intervals(NN=100, KK=2000, nIterations=4000, alpha_damp=1.5, seed=None):
    # Define the domain boundaries
    SMIN, SMAX = 10**2, 200**2
    COSTHETAMIN, COSTHETAMAX = -1, 1  

    # Initialize intervals with uniform spacing
    sLow = np.linspace(SMIN, SMAX, NN + 1)
    cosLow = np.linspace(COSTHETAMIN, COSTHETAMAX, NN + 1)

    # Initialize weights for intervals (uniform at start)
    weights_s = np.ones(NN)
    weights_cos = np.ones(NN)

    # Random number generator
    rng = np.random.RandomState(seed)

    # Iteratively adjust intervals based on function evaluations
    for _ in range(nIterations):
        # Randomly choose intervals based on current weights
        chosen_intervals_s = rng.choice(NN, size=KK, p=weights_s / weights_s.sum())
        chosen_intervals_cos = rng.choice(NN, size=KK, p=weights_cos / weights_cos.sum())

        # Sample within chosen intervals and evaluate the function
        samples_s = sLow[chosen_intervals_s] + rng.rand(KK) * (SMAX - SMIN) / NN
        samples_cos = cosLow[chosen_intervals_cos] + rng.rand(KK) * (COSTHETAMAX - COSTHETAMIN) / NN
        f_vals = diff_cross_section(samples_s, samples_cos)  # Your differential cross-section function

        # Adjust weights based on function evaluations and damping
        for i in range(NN):
            interval_mask_s = chosen_intervals_s == i
            interval_mask_cos = chosen_intervals_cos == i
            weights_s[i] *= alpha_damp + (f_vals[interval_mask_s].sum() / KK)
            weights_cos[i] *= alpha_damp + (f_vals[interval_mask_cos].sum() / KK)

        # Normalize weights to maintain the same total interval size
        weights_s /= weights_s.sum() / NN
        weights_cos /= weights_cos.sum() / NN

        # Recalculate interval boundaries based on new weights
        sLow[1:-1] = np.cumsum(weights_s[:-1]) * (SMAX - SMIN) / NN + SMIN
        cosLow[1:-1] = np.cumsum(weights_cos[:-1]) * (COSTHETAMAX - COSTHETAMIN) / NN + COSTHETAMIN
        
        delx = np.diff(sLow)
        dely = np.diff(cosLow)

    return sLow, cosLow, delx, dely

In [121]:
def vegas(nPoints, vegasRatioFactor, NN, KK, nIterations, alpha_damp, seed=None):
    # Setup initial intervals using the setup_intervals function
    COSTHETAMIN, COSTHETAMAX = -1, 1  
    
    sLow, cosLow, delx, dely = setup_intervals(NN, KK, nIterations, alpha_damp, seed)
    vegasRatioMax = vegasRatioFactor * F_VAL_MAX * NN * NN * delx[NN - 2] * dely[NN - 2]

    # Initialize variables
    nFunctionEval = 0
    s = []
    cos = []
    maxWeightEncountered = -1.0e20

    # Random number generator
    rng = np.random.RandomState(seed)

    # Main loop for sampling and evaluating
    while len(s) < nPoints:
        # Sample points within the refined intervals
        ixLow = rng.randint(0, NN)
        iyLow = rng.randint(0, NN)
        ss = sLow[ixLow] + delx[ixLow] * rng.uniform()
        cosT = cosLow[iyLow] + dely[iyLow] * rng.uniform()

        # Evaluate the differential cross-section at the sampled points
        f_vals = diff_cross_section(ss, cosT)
        ratio = f_vals * NN * NN * delx[ixLow] * dely[iyLow]
        nFunctionEval +=1

        # Update the maximum weight encountered, if necessary
        max_f_val = np.max(f_vals)
        if max_f_val > maxWeightEncountered:
            maxWeightEncountered = max_f_val

        # Store points based on the acceptance criterion
        # Accept the point if the Vegas ratio exceeds the vegasRatioFactor threshold
        if ratio / vegasRatioMax > rng.uniform():
            s.append(ss)
            cos.append(cosT)

    # Prepare the output
    output = (
        s,
        cos,
        nFunctionEval,
        maxWeightEncountered,
        vegasRatioFactor * max_f_val,  # Maximum Vegas ratio considered for acceptance
    )
    
    return output

s_vegas, cos_vegas, nFunctionalEval_vegas, maxweightEncounteredvegas, vegasRatioFactor = vegas(1000, 0.1, 50, 100, 2000, 4000, 1)
print(nFunctionalEval_vegas)

10663


In [122]:
import time
n_events = 1000
start_time_ar = time.time()
s_rej_method, cos_theta_rej_method, nFunctionEval_rej, maxWeightEncounteredReJ= brute_force(n_events)
end_time_ar = time.time()
time_ar = end_time_ar - start_time_ar

start_time_vegas = time.time()
s_vegas, cos_vegas, nFunctionalEval_vegas, maxweightEncounteredvegas,vegasRatioFactor = vegas(n_events,0.1,100,2000,4000,1.5)
end_time_vegas = time.time()
time_vegas = end_time_vegas - start_time_vegas

print("Acceptance-Rejection Method:")
print(f"Number of function evaluations: {nFunctionEval_rej}")
print(f"Time taken: {time_ar} seconds")

print("\nVEGAS Method:")
print(f"Time taken: {time_vegas} seconds")
print(f"Number of function evaluations: {nFunctionalEval_vegas}")

 f_val=6.687946878236332e-07 exceeds F_VAL_MAX=6.631486904825394e-07, program will now exit
 f_val=6.759619925492814e-07 exceeds F_VAL_MAX=6.631486904825394e-07, program will now exit
Acceptance-Rejection Method:
Number of function evaluations: 66677
Time taken: 1.996596097946167 seconds

VEGAS Method:
Time taken: 7.866574048995972 seconds
Number of function evaluations: 10219


In [56]:
import tqdm
events = [10, 100, 1000, 10000, 100000, 300000, 700000, 1000000]
ar_times = []
vegas_times = []
for i in enumerate(events):
    start_time_ar = time.time()
    events_ar, function_evaluations_ar = ar_generate_events(i[1])
    end_time_ar = time.time()
    time_ar = end_time_ar - start_time_ar
    ar_times.append(time_ar)

    start_time_vegas = time.time()
    events_vegas = vegas_generate_events(i[1])
    end_time_vegas = time.time()
    time_vegas = end_time_vegas - start_time_vegas
    vegas_times.append(time_vegas)
plt.plot(events,vegas_times)
plt.plot(events,ar_times)
plt.show()

KeyboardInterrupt: 

10
100
1000
10000
100000
300000
700000
1000000
